# Crawler - Crawling

**link를 찾는 것**이 가장 중요하다

search space 내에서 정보를 탐색하는 방법(인공지능)
- 어디로 움직일 지를 선택하는 방법(DFS/BFS)

```python
while 더이상 방문할 곳이 없을때까지 # 봇이 안 죽게끔
  URL Pool => 앞으로 봇이 방문해야 할 페이지들의 목록
  # 방문해도 될까? robotparser
  seed -> 이번에 봇이 방문할 링크
  request(seed) -> response -> 1) 다른 링크, 2) 페이지 내용 분석(Scraping)
  link 추출(link가 어디에 있는지, link 어떻게 생겼는지) # 많은 링크를 찾아야 함
  정규화(URL형태로)
  방문한 적이 있는지? #전략;
  # 다음번 방문 전, 적당한 delay
  없으면 URL Pool 추가
```

In [ ]:
# O -> ? (DFS, BFS)
#                  X  (검색결과)
#       X    X    X     X     X
#  X  O  O  O ...     어디가 끝인지 알 수 없음

In [ ]:
from requests import get
from requests.compat import urljoin, urlparse
import re

In [ ]:
# robotparser
# 도메인을 받아서,
# robots.txt 확인
# agent:
# [Dis]Allow: _______ 규칙
robots = dict()

def canFetch(host, path):
    if not re.search('^https?', host):
        return 'Scheme 오류'

    url = host+'/' if host[-1] != '/' else host
    url += 'robots.txt'

    k = urlparse(url).netloc
    if k not in robots.keys():
        resp = get(url)
        if resp.status_code != 200:
            return resp.status_code

        if k not in robots.keys():
            robots[k] = re.findall('^disallow:\s*(.+)$', resp.text, re.IGNORECASE|re.MULTILINE)

    if path in robots[k]:
        return False
    else:
        # path.split('/') # 나중에 완성
        return True

[' /']

In [ ]:
from time import sleep

URLs = list()
Seens = list()

URLs.append('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EC%B9%B4%EB%A6%AC%EB%82%98')
headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Whale/3.26.244.21 Safari/537.36'}

while URLs:
    seed = URLs.pop(0) # Queue
    Seens.append(seed)

    components = urlparse(seed)

    if canFetch('://'.join(components[:2]), components.path) == False:
        print('가져가면 안되요')

    resp = get(seed, headers=headers)

    if resp.status_code != 200:
        if resp.status_code == 500:
            URLs.append(seed)
        else:
            print('Response 없음')
            continue

    if not re.search(r'text/html', resp.headers['content-type']):
        continue

    dom = BeautifulSoup(resp.text, 'html.parser')
    for link in dom.select('a[href], iframe[src]'):
        href = link.attrs['href'] if link.has_attr('href') else link.attrs['src']
        if not re.match(r'#|javascript', href):
            nurl = urljoin(seed, href)
            if nurl not in Seens: # 한번이라도 방문한적이 없으면
                URLs.append(nurl)

    sleep(1) # <= 난수

In [ ]:
len(URLs), len(Seens)

In [ ]:
Seens

In [ ]:
seed, resp.headers, resp.status_code, resp.reason

In [ ]:
get(seed, headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Whale/3.26.244.21 Safari/537.36'})

In [ ]:
# BFS => [1], [2, 3, 4], [5, 6, 7] => Queue(FIFO)
# DFS => [1], [4] [7, 6, 5], [3] [2] => Stack(LIFO)
#               1
#       2       3       4
#                     5 6 7

In [ ]:
[1,2,3,4].pop(0), [1,2,3,4].pop(-1)

In [ ]:
    # <a href="">
    # <button>
    # <form>
    # <iframe src="">
    # <audio>
    # <video>